# Test GeoV-9b llm

This is a sample notebook to try the GeoV 9 billion parameter language model.

This notebook runs the the 9 billion model with [LLM.int8()](https://papers.labml.ai/paper/eb2bcaee1d0011edaa66a71c10a887e7) and uses only ~10GB GPU memory.

You can find the source repo at [geov-ai/geov](https://github.com/geov-ai/geov) and the model weights at [GeoV/GeoV-9b](https://huggingface.co/GeoV/GeoV-9b).

Install dependencies

In [1]:
%%capture

!pip install geov bitsandbytes accelerate

Import modules

In [2]:
import torch
from geov import GeoVForCausalLM, GeoVTokenizer

Load the GeoV model

This might take a few minutes because it needs to download the weights from [GeoV/GeoV-9b](https://huggingface.co/GeoV/GeoV-9b)

In [3]:
model = GeoVForCausalLM.from_pretrained('GeoV/GeoV-9b', load_in_8bit=True, device_map='auto')

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-2y59rlpiffdiq --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

Load the tokenizer

In [4]:
tokenizer = GeoVTokenizer.from_pretrained('GeoV/GeoV-9b')

Prompt

In [5]:
prompt = "In mathematics, topology is the study of"

Encode the prompt with the tokenizer

In [6]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

Generate

In [7]:
with torch.no_grad():
    gen_tokens = model.generate(
        input_ids=input_ids.to(torch.device('cuda:0')),
        do_sample=True,
        temperature=1.,
        max_length=256,
    )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Decode the generated output

In [8]:
tokenizer.batch_decode(gen_tokens)[0]

'In mathematics, topology is the study of shapes and the counting of shapes in a given shape space (or topological space).\nThe ability to interpret an entire topology theory in just a few pages, that explain complex and difficult concepts in clear, easy to understand text and pictures, makes this book a true reference for any (geometry/topology) student/ lecture as well as researcher in mathematics and computer science.\nThe study of shape (topology) in mathematics, computer science and in any other domain is extremely important nowadays. This book is a great help!\nVery concise (not too wordy) text and some really useful diagrams. Recommended!\nVery good explanation of a very hard (but also important) topic. I recommend the book to everyone studying mathematics and computer science.\nThe book is good for people who want to know basic ideas about topologies and how topology is related to geometric computing and computer science. The book does not pretend to be comprehensive; it attemp